In [73]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!tar -xvf spark-3.0.1-bin-hadoop3.2.tgz
!pip install -q findspark

spark-3.0.1-bin-hadoop3.2/
spark-3.0.1-bin-hadoop3.2/RELEASE
spark-3.0.1-bin-hadoop3.2/examples/
spark-3.0.1-bin-hadoop3.2/examples/src/
spark-3.0.1-bin-hadoop3.2/examples/src/main/
spark-3.0.1-bin-hadoop3.2/examples/src/main/scala/
spark-3.0.1-bin-hadoop3.2/examples/src/main/scala/org/
spark-3.0.1-bin-hadoop3.2/examples/src/main/scala/org/apache/
spark-3.0.1-bin-hadoop3.2/examples/src/main/scala/org/apache/spark/
spark-3.0.1-bin-hadoop3.2/examples/src/main/scala/org/apache/spark/examples/
spark-3.0.1-bin-hadoop3.2/examples/src/main/scala/org/apache/spark/examples/ml/
spark-3.0.1-bin-hadoop3.2/examples/src/main/scala/org/apache/spark/examples/ml/FPGrowthExample.scala
spark-3.0.1-bin-hadoop3.2/examples/src/main/scala/org/apache/spark/examples/ml/GBTExample.scala
spark-3.0.1-bin-hadoop3.2/examples/src/main/scala/org/apache/spark/examples/ml/ALSExample.scala
spark-3.0.1-bin-hadoop3.2/examples/src/main/scala/org/apache/spark/examples/ml/KMeansExample.scala
spark-3.0.1-bin-hadoop3.2/example

In [74]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

In [75]:
import findspark
findspark.init()
from pyspark import SparkConf, SparkContext

# spark = SparkSession.builder.master("local[*]").getOrCreate()

In [76]:
!pip install systemml

In [77]:
!pip install https://github.com/IBM/coursera/blob/master/systemml-1.3.0-SNAPSHOT-python.tar.gz?raw=true

  Using cached https://github.com/IBM/coursera/blob/master/systemml-1.3.0-SNAPSHOT-python.tar.gz?raw=true
  Created wheel for systemml: filename=systemml-1.3.0-cp36-none-any.whl size=9882974 sha256=cfb44bd8a8c3dd08ddd8518414cf13952a89e09cf76581b2d6839f32bc86ed5c
  Stored in directory: /root/.cache/pip/wheels/aa/bf/28/4344dd13abd8b9b6cbd4032baf4b851873d2e2288a65631fd2
Successfully built systemml


In [78]:
from systemml import MLContext
ml = MLContext(spark)
ml.version()

'1.3.0-SNAPSHOT'

In [79]:
from pyspark import SparkConf, SparkContext

In [80]:
conf = SparkConf().setMaster("local").setAppName("FriendsByAge")
sc = SparkContext.getOrCreate(conf=conf)


In [81]:
def parseLine(line):
    fields = line.split(',')
    age = int(fields[2])
    numFriends = int(fields[3])
    return (age, numFriends)

In [82]:
lines = sc.textFile("fakefriends.csv")
rdd = lines.map(parseLine)
# print(rdd.collect())
totalsByAge = rdd.mapValues(lambda x: (x, 1)).reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
# print(totalsByAge.collect())
averagesByAge = totalsByAge.mapValues(lambda x: x[0] / x[1])
results = averagesByAge.collect()
for result in results:
    print(result)


(26, 242.05882352941177)
(40, 250.8235294117647)
(68, 269.6)
(54, 278.0769230769231)
(38, 193.53333333333333)
(56, 306.6666666666667)
(36, 246.6)
(22, 206.42857142857142)
(60, 202.71428571428572)
(30, 235.8181818181818)
(42, 303.5)
(48, 281.4)
(50, 254.6)
(32, 207.9090909090909)
(58, 116.54545454545455)
(64, 281.3333333333333)
(52, 340.6363636363636)
(24, 233.8)
(20, 165.0)
(62, 220.76923076923077)
(44, 282.1666666666667)
(28, 209.1)
(66, 276.44444444444446)
(46, 223.69230769230768)
(18, 343.375)
(34, 245.5)
(33, 325.3333333333333)
(55, 295.53846153846155)
(59, 220.0)
(37, 249.33333333333334)
(27, 228.125)
(53, 222.85714285714286)
(57, 258.8333333333333)
(43, 230.57142857142858)
(35, 211.625)
(45, 309.53846153846155)
(67, 214.625)
(19, 213.27272727272728)
(51, 302.14285714285717)
(25, 197.45454545454547)
(21, 350.875)
(49, 184.66666666666666)
(39, 169.28571428571428)
(31, 267.25)
(41, 268.55555555555554)
(69, 235.2)
(65, 298.2)
(61, 256.22222222222223)
(29, 215.91666666666666)
(47, 233

In [83]:
conf = SparkConf().setMaster("local").setAppName("MinTemperatures")
# sc = SparkContext(conf = conf)
sc = SparkContext.getOrCreate(conf=conf)

In [84]:
def parseLine(line):
    fields = line.split(',')
    stationID = fields[0]
    entryType = fields[2]
    temperature = float(fields[3]) * 0.1 * (9.0 / 5.0) + 32.0
    return (stationID, entryType, temperature)

In [85]:
lines = sc.textFile("/content/1800.csv")
parsedLines = lines.map(parseLine)
print(parsedLines.collect())
minTemps = parsedLines.filter(lambda x: "TMIN" in x[1])
stationTemps = minTemps.map(lambda x: (x[0], x[2]))
print(stationTemps.collect())
minTemps = stationTemps.reduceByKey(lambda x, y: min(x,y))
maxTemps = stationTemps.reduceByKey(lambda x, y: max(x,y))
print(minTemps.collect())
min_results = minTemps.collect();
max_results = maxTemps.collect();

[('ITE00100554', 'TMAX', 18.5), ('ITE00100554', 'TMIN', 5.359999999999999), ('GM000010962', 'PRCP', 32.0), ('EZE00100082', 'TMAX', 16.52), ('EZE00100082', 'TMIN', 7.699999999999999), ('ITE00100554', 'TMAX', 21.2), ('ITE00100554', 'TMIN', 9.5), ('GM000010962', 'PRCP', 32.0), ('EZE00100082', 'TMAX', 24.08), ('EZE00100082', 'TMIN', 8.599999999999998), ('ITE00100554', 'TMAX', 27.86), ('ITE00100554', 'TMIN', 23.72), ('GM000010962', 'PRCP', 32.72), ('EZE00100082', 'TMAX', 30.2), ('EZE00100082', 'TMIN', 18.86), ('ITE00100554', 'TMAX', 32.0), ('ITE00100554', 'TMIN', 29.66), ('GM000010962', 'PRCP', 32.0), ('EZE00100082', 'TMAX', 22.1), ('EZE00100082', 'TMIN', 18.68), ('ITE00100554', 'TMAX', 33.8), ('ITE00100554', 'TMIN', 30.919999999999998), ('GM000010962', 'PRCP', 32.0), ('EZE00100082', 'TMAX', 24.8), ('EZE00100082', 'TMIN', 21.56), ('ITE00100554', 'TMAX', 34.34), ('ITE00100554', 'TMIN', 34.34), ('GM000010962', 'PRCP', 32.0), ('EZE00100082', 'TMAX', 24.98), ('EZE00100082', 'TMIN', 21.740000000

In [86]:
for result in min_results:
    print(result[0] + "\t{:.2f}F".format(result[1]))
for result in max_results:
    print(result[0] + "\t{:.2f}F".format(result[1]))

ITE00100554	5.36F
EZE00100082	7.70F
ITE00100554	75.38F
EZE00100082	68.72F


In [87]:
input = sc.textFile("/content/Book.txt")
words = input.flatMap(lambda x: x.split())
wordCounts = words.countByValue()

for word, count in wordCounts.items():
    cleanWord = word.encode('ascii', 'ignore')
    if (cleanWord):
        print(cleanWord.decode() + " " + str(count))

Streaming output truncated to the last 5000 lines.
individual. 2
top-down 1
restrictions 2
make, 1
clients. 2
computer 6
programmer, 1
conform 1
style 4
guide 1
existing 17
architecture 1
artist, 1
show 16
Working 4
what's 6
important. 8
Did 2
dress 1
code 3
yourself? 1
meeting 2
clients, 2
want. 9
maintain 9
wardrobe. 1
Personally, 2
regardless; 1
helps 4
frame 1
mind. 5
IS 1
SELF-EMPLOYMENT 1
FOR 3
YOU? 1
Self-employment 4
forms 4
cannot 9
environment. 1
unbounded 1
rewards, 2
far, 5
serious. 1
Let's 8
you? 4
answer. 1
honestly 2
assess 1
diving 2
undertaking, 1
likelihood 2
success. 8
then, 1
guarantees 1
exercises 1
chapter 1
stand. 1
unimaginably 1
well, 11
badly. 1
We 6
favor 4
not, 13
conventional 2
workforce 1
savings 7
now. 6
FLOWCHART: 1
SHOULD 1
EVEN 1
CONSIDER 1
SELF-EMPLOYMENT? 1
Minimizing 1
ensuring 3
cushion 1
expenses. 6
truly 3
betting 1
farm 5
identify 8
store 1
changes. 2
flowchart 2
crazy 1
this. 2
reading 6
device 1
display 17
main 6
points: 1
HAVING 1
FINANCIAL 2

In [91]:
import re
def normalizeWords(text):
    return re.compile(r'\W+', re.UNICODE).split(text.lower())

In [92]:
input = sc.textFile("/content/Book.txt")
words = input.flatMap(normalizeWords)
print(words.collect())

['self', 'employment', 'building', 'an', 'internet', 'business', 'of', 'one', 'achieving', 'financial', 'and', 'personal', 'freedom', 'through', 'a', 'lifestyle', 'technology', 'business', 'by', 'frank', 'kane', '', '', '', 'copyright', '2015', 'frank', 'kane', '', 'all', 'rights', 'reserved', 'worldwide', '', '', '', 'contents', 'disclaimer', 'preface', 'part', 'i', 'making', 'the', 'big', 'decision', 'overcoming', 'inertia', 'fear', 'of', 'failure', 'career', 'indoctrination', 'the', 'carrot', 'on', 'a', 'stick', 'ego', 'protection', 'your', 'employer', 'as', 'a', 'security', 'blanket', 'why', 'it', 's', 'worth', 'it', 'unlimited', 'growth', 'potential', 'investing', 'in', 'yourself', 'not', 'someone', 'else', 'no', 'dependencies', 'no', 'commute', 'freedom', 'to', 'live', 'where', 'you', 'want', 'freedom', 'to', 'work', 'when', 'you', 'want', 'freedom', 'to', 'work', 'how', 'you', 'want', 'is', 'self', 'employment', 'for', 'you', '', 'flowchart', 'should', 'i', 'even', 'consider', '

In [93]:
wordCounts = words.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)
wordCountsSorted = wordCounts.map(lambda x: (x[1], x[0])).sortByKey()
results = wordCountsSorted.collect()

In [94]:
for result in results:
    count = str(result[0])
    word = result[1].encode('ascii', 'ignore')
    if (word):
        print(word.decode() + ":\t\t" + count)

achieving:		1
contents:		1
preparation:		1
skillset:		1
determination:		1
blame:		1
devoted:		1
commuted:		1
rewarded:		1
marriage:		1
ultimatum:		1
weeks:		1
walked:		1
nor:		1
heart:		1
smarts:		1
rebel:		1
magnitude:		1
justify:		1
surprising:		1
starving:		1
quits:		1
tenure:		1
americans:		1
14:		1
fringe:		1
internal:		1
religious:		1
questioning:		1
instill:		1
grades:		1
graduated:		1
absorbed:		1
promotes:		1
barely:		1
fulfill:		1
cup:		1
ramen:		1
noodles:		1
paths:		1
commensurate:		1
banking:		1
materially:		1
rationally:		1
tiny:		1
proceeds:		1
answered:		1
awards:		1
doubting:		1
difficulty:		1
promoted:		1
quantify:		1
mitigate:		1
suite:		1
millionaire:		1
liked:		1
formed:		1
guts:		1
miles:		1
multiple:		1
filled:		1
paralegal:		1
hassle:		1
switch:		1
super:		1
hats:		1
css:		1
similarly:		1
passive:		1
disproportionate:		1
promote:		1
ratio:		1
faceless:		1
returns:		1
cooperation:		1
derail:		1
inaction:		1
68:		1
hinges:		1
fault:		1
scares:		1
mile:		1
17:		1
y

In [95]:
def extractCustomerPricePairs(line):
    fields = line.split(',')
    return (int(fields[0]), float(fields[2]))

In [96]:
input = sc.textFile("/content/customer-orders.csv")
mappedInput = input.map(extractCustomerPricePairs)
totalByCustomer = mappedInput.reduceByKey(lambda x, y: x + y)

In [98]:
results = totalByCustomer.collect();
for result in results:
    print(result)

(44, 4756.890000000001)
(2, 5994.59)
(70, 5368.249999999999)
(14, 4735.030000000001)
(42, 5696.840000000002)
(50, 4517.2699999999995)
(20, 4836.860000000001)
(48, 4384.33)
(4, 4815.050000000001)
(36, 4278.049999999999)
(12, 4664.589999999999)
(22, 5019.449999999999)
(54, 6065.390000000001)
(0, 5524.949999999999)
(88, 4830.55)
(86, 4908.810000000001)
(40, 5186.429999999999)
(98, 4297.259999999999)
(78, 4524.51)
(6, 5397.879999999999)
(26, 5250.4)
(60, 5040.709999999999)
(64, 5288.689999999999)
(82, 4812.49)
(72, 5337.439999999999)
(34, 5330.8)
(8, 5517.24)
(46, 5963.109999999999)
(80, 4727.86)
(16, 4979.06)
(18, 4921.269999999999)
(74, 4647.130000000001)
(30, 4990.72)
(56, 4701.02)
(90, 5290.41)
(68, 6375.450000000001)
(10, 4819.7)
(58, 5437.73)
(62, 5253.3200000000015)
(92, 5379.279999999999)
(76, 4904.210000000001)
(66, 4681.92)
(52, 5245.0599999999995)
(84, 4652.9400000000005)
(24, 5259.92)
(96, 3924.2300000000005)
(94, 4475.570000000001)
(32, 5496.05)
(38, 4898.460000000001)
(28, 50

In [99]:
mappedInput = input.map(extractCustomerPricePairs)
totalByCustomer = mappedInput.reduceByKey(lambda x, y: x + y)

In [100]:
flipped = totalByCustomer.map(lambda x: (x[1], x[0]))

In [101]:
totalByCustomerSorted = flipped.sortByKey()
results = totalByCustomerSorted.collect();
for result in results:
    print(result)

(3309.3799999999997, 45)
(3790.5699999999997, 79)
(3924.2300000000005, 96)
(4042.65, 23)
(4172.29, 99)
(4178.5, 75)
(4278.049999999999, 36)
(4297.259999999999, 98)
(4316.299999999998, 47)
(4327.73, 77)
(4367.619999999999, 13)
(4384.33, 48)
(4394.6, 49)
(4475.570000000001, 94)
(4505.79, 67)
(4517.2699999999995, 50)
(4524.51, 78)
(4561.07, 5)
(4628.4, 57)
(4635.8, 83)
(4642.26, 91)
(4647.130000000001, 74)
(4652.9400000000005, 84)
(4659.63, 3)
(4664.589999999999, 12)
(4681.92, 66)
(4701.02, 56)
(4707.41, 21)
(4727.86, 80)
(4735.030000000001, 14)
(4735.200000000001, 37)
(4755.069999999999, 7)
(4756.890000000001, 44)
(4765.049999999999, 31)
(4812.49, 82)
(4815.050000000001, 4)
(4819.7, 10)
(4830.55, 88)
(4836.860000000001, 20)
(4851.48, 89)
(4876.839999999998, 95)
(4898.460000000001, 38)
(4904.210000000001, 76)
(4908.810000000001, 86)
(4915.89, 27)
(4921.269999999999, 18)
(4945.300000000001, 53)
(4958.600000000001, 1)
(4975.220000000001, 51)
(4979.06, 16)
(4990.72, 30)
(5000.709999999999, 2